In [1]:
import streamlit as st
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
from itertools import combinations
from decimal import Decimal

pd.set_option('display.max_columns', None)

In [2]:
def caminho_absoluto(caminho_relativo_com_barras_normais):
    
    caminho_base = os.getcwd()

    caminho_absoluto = os.path.join(caminho_base, caminho_relativo_com_barras_normais)

    return caminho_absoluto

In [3]:
situacao_final = pd.read_excel(caminho_absoluto('data/tratamento_curva_abc/dados_tratados/situacao_final.xlsx')).set_index('Ordem')
local_frac = pd.read_excel(caminho_absoluto('data/analise_curva_abc/local/datasets/local_apanha_frac.xlsx'))    

In [4]:
situacao_final.iloc[[0]]

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,106011,NARIDRIN 12 HORAS FR C/30ML,A,A,A,279389.0,6507.0,930089.0,271.0,251.0,274.0,1490.0,164.0,1490.0,1030.9557,25.9243,3394.4854,5057,016-037-01-02,50,2,100,387,500,Não,29.913,Ponta De G,3236.0,pallet


In [5]:
local_frac.iloc[[0]]

,Ender.Fracionado,modulo,local
0,29.011,1,AB


In [26]:
AC = 20
AB = 32
AA = 27
AM = 8
XPE = 9

total_enderecos_mosdulo = AC + AB + AA

numero_modulos = 5

total_enderecos_molulos = numero_modulos * total_enderecos_mosdulo
total_enderecos_aa = numero_modulos * AA
total_enderecos_ab = numero_modulos * AB
total_enderecos_ac = numero_modulos * AC
total_enderecos_am = numero_modulos * AM
total_enderecos_xpe = numero_modulos * XPE

In [27]:
#### Seleção e tabela de itens errados (curva A no flowrack - mudar para a prateleira [itens cuja ordem é mair o do que a quantidade de endereços do flowrack])

#Itens de curva A, que são medicamentos normais
selecao_med_normal = (~situacao_final['Descrição'].str.contains('xpe|susp|sol|elixir', case=False)) & \
                     (~situacao_final['Descrição'].str.contains('SPRAYZIIN|escova|ABS|DENTAL|TOALHAS|AG ABSORVENTE|COMPRESSA|AG MULTIFRAL|AG PANTS|FITA|MASCARA|ATADURA|CERA ORTODONTICA|ESPARADRAPO', case=False)) & \
                     (situacao_final['Ender.Fracionado'] > 12.999) & \
                     (situacao_final['Curva Frac'] == 'A') & \
                     (situacao_final['local'] != 'alimento')

somente_med_A = situacao_final[selecao_med_normal]
total_somente_med_A = somente_med_A.shape[0]

num_linhas_a_excluir = total_somente_med_A - total_enderecos_molulos

if total_somente_med_A > total_enderecos_molulos:
    num_total_linhas = len(somente_med_A)
    produtos_para_flowrack = somente_med_A.drop(somente_med_A.tail(num_linhas_a_excluir).index)
    linhas_excluidas = somente_med_A.tail(num_linhas_a_excluir)
else:
    produtos_para_flowrack = somente_med_A
    linhas_excluidas = []

total_produtos_para_flowrack = produtos_para_flowrack.shape[0]

#Medicamentos curva A normais que estão na prateleira e devem ir para o flowrack
produtos_na_prateleira = situacao_final[situacao_final['Tipo'] == 'Prateleira']
selecao_curva_a_normal_prateleira_para_flowrack = (produtos_na_prateleira['Código']).isin(produtos_para_flowrack['Código'])

curva_a_normal_prateleira_para_flowrack = produtos_na_prateleira[selecao_curva_a_normal_prateleira_para_flowrack]

total_curva_a_normal_prateleira_para_flowrack = curva_a_normal_prateleira_para_flowrack.shape[0]

#Medicamentos curva A normais que estão no flowrack e devem ir para a prateleira
produtos_no_flowrack = situacao_final[situacao_final['Tipo'] == 'Flowrack']
selecao_curva_a_normal_flowrack_para_prateleira = (produtos_no_flowrack['Código']).isin(linhas_excluidas['Código'])

curva_a_normal_flowrack_para_prateleira = produtos_no_flowrack[selecao_curva_a_normal_flowrack_para_prateleira]

total_curva_a_normal_flowrack_para_prateleira = curva_a_normal_flowrack_para_prateleira.shape[0]

In [32]:
produtos_para_classe_AA = produtos_para_flowrack[0 : (total_enderecos_aa)]

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,106011,NARIDRIN 12 HORAS FR C/30ML,A,A,A,279389.0,6507.0,930089.0,271.0,251.0,274.0,1490.0,164.0,1490.0,1030.9557,25.9243,3394.4854,5057,016-037-01-02,50,2,100,387,500,Não,29.913,Ponta De G,3236.0,pallet
4,102011,CYSTEX C/ 24DRG,A,A,A,121411.0,251.0,146511.0,280.0,51.0,280.0,1011.0,283.0,1011.0,433.6107,4.9216,523.2536,217,014-022-02-02,13,20,100,242,300,Não,29.063,Flowrack,4911.0,apanha_a
6,101175,INFRALAX C/ 15 COMP,A,A,A,98494.0,1309.0,161326.0,270.0,197.0,271.0,779.0,280.0,779.0,364.7926,6.6447,595.2989,450,014-032-02-01,30,30,48,328,332,Não,20.083,Flowrack,1983.0,apanha_a
7,231064,REPOFLOR 200MG C/ 06CAPS,A,A,A,94594.0,268.0,121394.0,264.0,49.0,264.0,701.0,225.0,701.0,358.3106,5.4694,459.8258,273,014-034-01-02,10,15,100,232,300,Não,25.012,Flowrack,2242.0,apanha_a
8,102138,INFRALAX C/30 COMP,A,A,A,87301.0,1290.0,149221.0,267.0,193.0,268.0,605.0,263.0,605.0,326.9700,6.6839,556.7948,328,014-042-02-01,36,30,48,369,384,Não,22.032,Flowrack,1832.0,apanha_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186,11361,CETOPROFENO 150MG C/ 10COMP LIB PROL,A,B,A,5426.0,34.0,7058.0,211.0,16.0,211.0,40.0,49.0,40.0,25.7156,2.1250,33.4502,0,014-012-02-01,1,6,48,163,200,Não,23.014,Flowrack,315.0,apanha_a
187,11314,TADALAFILA 20MG C/ 4COMP,A,C,A,5392.0,3.0,5680.0,109.0,3.0,109.0,88.0,31.0,88.0,49.4679,1.0000,52.1101,0,014-026-01-02,0,3,96,0,300,Não,20.023,Flowrack,222.0,apanha_a
189,231047,ESPASMO FLATOL C/ 20ML,A,C,A,5330.0,1.0,5450.0,238.0,1.0,238.0,45.0,23.0,45.0,22.3950,1.0000,22.8992,1,014-032-01-02,3,4,120,187,200,Não,21.024,Flowrack,274.0,apanha_a


In [35]:
produtos_para_classe_AB = produtos_para_flowrack[total_enderecos_aa: (total_enderecos_aa + total_enderecos_ab)]

,Código,Descrição,Curva Frac,Curva Cx,Curva Geral,Qtde Venda Frac,Qtde Venda Cx,Qtde Venda Geral,Dias Pedido Frac,Dias Pedido Cx,Dias Pedido Geral,Ativ.Ressupr.Frac,Ativ.Ressupr.Cx,Ativ.Ressupr.Geral,Média por dia frac,Média por dia cx,Média por dia geral,Estoque Armaz.,Ender.Cx.Fechada,Estoque Cx,Capacidade Cx,Embal.,Estoque Frac,Capacidade Frac,Permite Frac.,Ender.Fracionado,Tipo,Qtde Desvio Picking,local
Ordem,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
195,11189,"DROSP+ETINIL 3MG+0,03MG C/21 COMP",A,B,A,5152.0,11.0,5680.0,213.0,6.0,214.0,47.0,32.0,47.0,24.1878,1.8333,26.5421,0,014-012-01-02,0,9,48,0,336,Não,28.022,Flowrack,367.0,apanha_a
196,10938,"INDAPAMIDA 1,5MG C/ 30COMP",A,B,A,5025.0,4.0,5217.0,181.0,4.0,181.0,38.0,27.0,38.0,27.7624,1.0000,28.8232,0,015-003-01-02,0,11,48,0,332,Não,28.082,Flowrack,322.0,apanha_a
197,460000,A SAUDE DA MULHER 150ML,A,B,A,4991.0,20.0,5691.0,238.0,15.0,239.0,136.0,126.0,136.0,20.9706,1.3333,23.8117,18,010-028-01-02,3,3,35,57,35,Não,26.903,Ponta De G,278.0,ponta
198,34045,GLICLAZIDA 60MG C/ 60 COM LIB PROL,A,B,A,4960.0,15.0,5800.0,164.0,8.0,164.0,241.0,25.0,241.0,30.2439,1.8750,35.3659,1,014-016-01-02,2 (+46 fr),4,56,42,50,Sim,23.043,Flowrack,108.0,apanha_a
199,106023,EMSCORT POMADA 30G,A,B,A,4921.0,8.0,5257.0,250.0,7.0,250.0,74.0,62.0,74.0,19.6840,1.1429,21.0280,8,015-037-01-01,3,6,42,179,140,Não,20.074,Flowrack,312.0,apanha_a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
433,10470,DIPROP BETAMETASONA CR 30G,A,C,A,1937.0,2.0,2021.0,198.0,2.0,198.0,21.0,20.0,21.0,9.7828,1.0000,10.2071,0,014-006-01-02,0,9,42,65,252,Não,23.061,Flowrack,209.0,apanha_a
436,101114,DOMPLIV 10MG C/ 60COMP,A,C,A,1924.0,1.0,2008.0,173.0,1.0,173.0,16.0,17.0,16.0,11.1214,1.0000,11.6069,0,015-003-01-01,0,5,84,238,330,Não,20.033,Flowrack,71.0,apanha_a
438,419432,ATENOLOL 25MG C/ 30COMP SANDOZ,A,B,A,1911.0,7.0,3171.0,54.0,1.0,55.0,19.0,10.0,19.0,35.3889,7.0000,57.6545,0,014-016-02-01,0,1,180,0,190,Sim,25.011,Flowrack,30.0,apanha_a


In [31]:
produtos_para_classe_AC = produtos_para_flowrack[(total_enderecos_aa + total_enderecos_ab) : (total_enderecos_aa + total_enderecos_ab + total_enderecos_ac)]

100

In [38]:
local_frac

,Ender.Fracionado,modulo,local
0,29.011,1,AB
1,29.012,1,AA
2,29.013,1,AA
3,29.014,1,AA
4,29.015,1,XPE
...,...,...,...
571,18.082,6,AA
572,18.083,6,AA
573,18.084,6,AA
574,18.085,6,XPE


In [44]:
local='AA'

selecao_do_local = local_frac['local'] == local

endereco_local_selec = local_frac[selecao_do_local]['Ender.Fracionado']


In [45]:
selecao_produtos_enderecado_no_local_selec = situacao_final['Ender.Fracionado'].isin(endereco_local_selec)

Ordem
1       False
2       False
3       False
4        True
5       False
        ...  
9654    False
9655    False
9656    False
9657    False
9658    False
Name: Ender.Fracionado, Length: 9658, dtype: bool